<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleMinst/T2E1_Kaggle_MINST_CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd 
import numpy as np
import tensorflow as tf

In [33]:
# Download Data
csv_train_file = '/content/drive/MyDrive/DS_data/MINSTHW/train.csv'
csv_test_file = '/content/drive/MyDrive/DS_data/MINSTHW/test.csv'

df_data = pd.read_csv(csv_train_file)
df_test = pd.read_csv(csv_test_file)

df_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Preprcessing

In [34]:
# Split train data to train and valid dataset
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

train_labels = df_train.pop('label')
valid_labels = df_valid.pop('label')

df_train.head()

#np.apply_along_axis(lambda x : np.reshape, 1, df_train.to_numpy(), 28, 28)
df_train = df_train.to_numpy().reshape(-1, 28 ,28, 1)
df_valid = df_valid.to_numpy().reshape(-1, 28 ,28, 1)

lenght of train is 41580
lenght of valid is 420


## Build the model

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)              

### Training

In [37]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(df_train, 
                    train_labels,
                    batch_size=64,
                    epochs=30)

Epoch 1/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0579 - accuracy: 0.9822
Epoch 2/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0574 - accuracy: 0.9820
Epoch 3/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0492 - accuracy: 0.9854
Epoch 4/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0478 - accuracy: 0.9853
Epoch 5/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0479 - accuracy: 0.9852
Epoch 6/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0442 - accuracy: 0.9857
Epoch 7/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0389 - accuracy: 0.9870
Epoch 8/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0386 - accuracy: 0.9874
Epoch 9/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0398 - accuracy: 0.9876
Epoch 10/50
650/650 [==============================] - 2s 3ms/step - loss: 0.0343 - accuracy: 0.9885

### Validate

In [39]:
 val_loss, val_acc = model.evaluate(df_valid,  valid_labels, verbose=2)

14/14 - 0s - loss: 0.0294 - accuracy: 0.9905


## Make predictions

In [40]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(df_test.to_numpy().reshape(-1, 28, 28, 1))

In [41]:
res = pd.DataFrame(data = {"ImageId" : np.arange(1, 1+len(predictions)) , 
                           "Label"   : list(map(np.argmax, predictions))})

res.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [42]:
res.to_csv("/content/drive/MyDrive/output/Kaggle_minst_hw_cnn2.csv", index=False)